In [2]:
import os
# Get access to google drive folder.
# This only needs to be done once per notebook.
from google.colab import drive
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/ACE/'  #change dir to your project folder

# Change current directoty to mounted on gdrive
os.chdir(root_path)
os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


'/content/gdrive/My Drive/ACE'

In [1]:
!ls datasets/Isophonics/

ls: cannot access 'datasets/Isophonics/': No such file or directory


In [0]:
# Number of semitones to deform by
semitones = 6

# Output format for audio (ogg, flac, etc)
# audio_ext = "ogg"
audio_ext = "flac"

# Output format for annotations (jams, jamz)
jams_ext = "jams"

# Path for input (audio, jams) pairs
input_path = "datasets/Isophonics/Queen"
# input_path = "datasets/Isophonics/Carole King"

# Path to store augmented data
output_path = "augment_results_collection/augment_results_queen"
# output_path = "augment_results_carole_king"

In [5]:
!ls datasets/Isophonics/

'Carole King'   Queen  'The Beatles'


In [0]:
def make_muda(n_semitones):
    '''Construct a MUDA pitch shifter'''

    tones = []
    # -6 to +5 semitones
    for n in range(-n_semitones, n_semitones):
        if n != 0:
            tones.append(n)

    shifter = muda.deformers.PitchShift(n_semitones=tones)

    smkdirs(output_path)
    with open(os.path.join(output_path, 'muda.pkl'), 'wb') as fd:
        pickle.dump(shifter, fd)

    return shifter

def augment(afile, jfile, deformer, outpath, audio_ext, jams_ext):
    '''Run the data through muda'''
    jam = muda.load_jam_audio(jfile, afile)

    base = crema_utils.base(afile)

    outfile = os.path.join(outpath, base)
    for i, jam_out in enumerate(deformer.transform(jam)):
        muda.save('{}.{}.{}'.format(outfile, i, audio_ext),
                  '{}.{}.{}'.format(outfile, i, jams_ext),
                  jam_out, strict=False)

In [10]:
try:
    from jams.util import smkdirs
except ImportError:
    !pip install -q jams
    from jams.util import smkdirs

try:
    from librosa.util import find_files
except ImportError:
    !pip install -q librosa==0.6.2
    from librosa.util import find_files

try:
    import soundfile
except ImportError:
    !pip install -q soundfile

try:
    import jsonpickle
except ImportError:
    !pip install -q jsonpickle

try:
    import muda
except ImportError:
    !pip install -q muda
    import muda
!dpkg -i rubberband-cli_1.8.1-4_amd64.deb
import pickle
from tqdm import tqdm
from crema import utils as crema_utils
from joblib import Parallel, delayed

smkdirs(output_path)

# Build the deformer
deformer = make_muda(n_semitones=semitones)

# Get the file list
loop = tqdm(crema_utils.get_ann_audio(input_path), desc='Augmenting training data')

for aud, ann  in loop:
    augment(aud, ann, deformer, output_path, audio_ext, jams_ext)
    print(aud)


(Reading database ... 131188 files and directories currently installed.)
Preparing to unpack rubberband-cli_1.8.1-4_amd64.deb ...
Unpacking rubberband-cli (1.8.1-4) over (1.8.1-4) ...
Setting up rubberband-cli (1.8.1-4) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


Augmenting training data:   0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: ignored